In [ ]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras import models
from tensorflow.keras import layers

import datetime
import tensorflow_addons as tfa 
import pandas as pd
import matplotlib.pylab as plt
import numpy as np

## Load data

In [ ]:
import read_rijksdata
NUM = 10000 # NUMBER OF IMAGES TO USE
MIN_ARTWORK_NUM = 10

In [ ]:
X_train, Y_train, label_train, X_val, Y_val, label_val = read_rijksdata.load_data(img_folder='out_img',
                                                                                  labels_file='labels.txt', 
                                                                                  names_file='names.txt')

In [ ]:
if NUM:
    X = X_train[:NUM,:,:,:]
    Y = Y_train[:NUM]
else:
    X = X_train
    Y = Y_train


In [ ]:
names = pd.read_csv('names.txt',delimiter = '/t',header=None,engine='python')
counts = pd.DataFrame(Y).value_counts()
min_counts = counts[counts>=MIN_ARTWORK_NUM]
min_idx = [index[0] for index in min_counts.index.tolist()]

min_mask_idx = []
for i, Y_ in enumerate(Y):
    if np.isin(Y_,min_idx):
        min_mask_idx.append(i)

In [ ]:
X_ = X[min_mask_idx,:,:,:]
Y_ = Y[min_mask_idx]


In [ ]:
classes = len(list(set(Y_)))
print(classes)

counts = pd.DataFrame(Y_).value_counts()
print(min(counts))

print(X_.shape)
print(Y_.shape)

In [ ]:
classes = len(list(set(Y_)))
Y_hot = tf.one_hot(indices=Y_,depth=classes)
#Y_val = tf.one_hot(indices=Y_val,depth=classes)
print('# of classes:',classes)

## Download and compile model

In [ ]:
input_shape = (56,56,3)
enet_kwargs = {'include_top':False,
               'weights':'imagenet',
               'input_tensor':None,
               'input_shape':input_shape,
               'pooling':None,
               'classes':1,
               'classifier_activation':'softmax'}
enet_base = tf.keras.applications.efficientnet.EfficientNetB7(**enet_kwargs)
#enet_base.trainable = False

In [ ]:
enet = tf.keras.models.Sequential()
enet.add(enet_base)
enet.add(tf.keras.layers.GlobalMaxPooling2D())
#enet.add(tf.keras.layers.Dropout(rate=0.01))
enet.add(tf.keras.layers.Dense(classes, activation="softmax"))

In [ ]:
enet.summary()

In [ ]:
# loss function
loss = tf.keras.losses.CategoricalCrossentropy(from_logits=False,label_smoothing=0.0,name='categorical_crossentropy')

# metrics

TopKs = []
for k in [1,5]:
    TopK = tf.keras.metrics.TopKCategoricalAccuracy(k=k, name='top_{}'.format(k))
    TopKs.append(TopK)
metrics = ["acc"]
metrics.extend(TopKs)

f1 = tfa.metrics.F1Score(num_classes=classes, threshold=0.05)
metrics.append(f1)

# Optimizer
# very average Adam settings
optimizer = tf.keras.optimizers.Adam(learning_rate=0.001)

# compile it all
enet.compile(
    loss=loss,
    optimizer=optimizer,
    metrics=metrics)

## Train model

In [ ]:
print(Y_hot.shape, type(Y_hot))
print(X_.shape, type(X))

In [ ]:
history = enet.fit(x=X_,y=Y_hot,validation_split=.20, epochs=10)

In [ ]:
# Save the weights
checkpoint_time = '{date:%Y-%m-%d_%H-%M}'.format(date=datetime.datetime.now())
save_file = './checkpoints/enet_{}'.format(checkpoint_time)
print('Saving to:',save_file)
enet.save_weights(save_file)

## Evaluation

In [ ]:
y_pred = enet.predict(X)

In [ ]:
from sklearn.preprocessing import label_binarize
from sklearn.metrics import roc_curve, auc
from sklearn.metrics import roc_auc_score
from sklearn.metrics import confusion_matrix

confusion_matrix = sklearn.metrics.confusion_matrix(y_true=Y, y_pred=y_pred, labels=None)

In [ ]:
fpr = dict()
tpr = dict()
roc_auc = dict()
for i in range(classes):
    fpr[i], tpr[i], _ = roc_curve(Y_train, y_pred)
    roc_auc[i] = auc(fpr[i], tpr[i])